# TreeDex Demo — Agentic RAG with Groq

**Tree-based, vectorless agentic RAG** — index any document, get direct answers with source pages.

This notebook:
1. Installs TreeDex from PyPI
2. Connects to **Groq** (free, fast inference)
3. Uploads a PDF and builds a tree index
4. Queries with **agentic mode** — gets direct answers, not just context

---

## 1. Install TreeDex

In [ ]:
!pip install -q treedex groq

In [ ]:
import treedex
print(f"TreeDex v{treedex.__version__} installed!")

## 2. Setup Groq LLM

1. Get a free API key at: https://console.groq.com/keys
2. In Colab sidebar, click the **key icon** (Secrets)
3. Add a secret named `GROQ_API_KEY` with your key
4. **Toggle "Notebook access" ON** for that secret

In [ ]:
from treedex import GroqLLM

try:
    from google.colab import userdata
    GROQ_KEY = userdata.get("GROQ_API_KEY")
    print(f"Key loaded from Colab secrets ({len(GROQ_KEY)} chars)")
except Exception as e:
    print(f"Could not load from secrets: {e}")
    GROQ_KEY = input("Paste your Groq API key: ").strip()

llm = GroqLLM(api_key=GROQ_KEY)
print(f"Using: {llm}")

In [ ]:
# Test the API key
try:
    response = llm.generate("Say 'hello' in one word.")
    print(f"API test passed! Response: {response}")
except Exception as e:
    print(f"API test FAILED: {e}")
    print("\nGet a free key at: https://console.groq.com/keys")

## 3. Upload a PDF

In [ ]:
from google.colab import files as colab_files

uploaded = colab_files.upload()
pdf_name = list(uploaded.keys())[0]
print(f"Uploaded: {pdf_name}")

## 4. Build the Index

In [ ]:
from treedex import TreeDex

index = TreeDex.from_file(pdf_name, llm=llm)

## 5. Inspect

In [ ]:
index.show_tree()

In [ ]:
stats = index.stats()
for k, v in stats.items():
    print(f"  {k}: {v}")

## 6. Save & Load

In [ ]:
index.save("my_index.json")
print("Saved!")

index2 = TreeDex.load("my_index.json", llm=llm)
print(f"Loaded: {index2.stats()['total_nodes']} nodes")

## 7. Query (Standard Mode)

In [ ]:
result = index.query("What are the main topics covered in this document?")

print(f"Nodes: {result.node_ids}")
print(f"Source: {result.pages_str}")
print(f"Reasoning: {result.reasoning}")
print(f"\nContext ({len(result.context)} chars):")
print(result.context[:1000])

In [ ]:
result2 = index.query("Explain the key concepts in the first chapter.")

print(f"Nodes: {result2.node_ids}")
print(f"Source: {result2.pages_str}")
print(f"Reasoning: {result2.reasoning}")

## 8. Agentic RAG Mode

With `agentic=True`, TreeDex retrieves relevant sections **and** generates a direct answer using the LLM.

In [ ]:
# Try your own question
q = input("Your question: ")
r = index.query(q)
print(f"\nSource: {r.pages_str}")
print(f"Reasoning: {r.reasoning}")
print(f"\n{r.context[:1500]}")

In [ ]:
# Agentic query — get a direct answer
result = index.query("What are the main topics covered?", agentic=True)

print(f"Answer:\n{result.answer}")
print(f"\nSource: {result.pages_str}")
print(f"Nodes: {result.node_ids}")

In [ ]:
# Interactive agentic query
q = input("Your question: ")
r = index.query(q, agentic=True)
print(f"\nAnswer:\n{r.answer}")
print(f"\nSource: {r.pages_str}")
print(f"Reasoning: {r.reasoning}")